In [ ]:
import os
os.chdir("/data/projects/punim2016/zexianh/LMSeg")

import pyvista as pv
pv.start_xvfb()  

w_size = (1000, 1000)
zoom = 1.3

import torch
import torch.nn.functional as F
import numpy as np
from data.dataset import H3DDataset

h3d_set = H3DDataset(root='data/H3DBenchmark/', split='val', epoch='Epoch_March2018')
idx = 1

mesh = pv.read(h3d_set.processed_mesh_list[idx])

face_v = h3d_set[idx].pos.cpu().numpy()
face_adj = h3d_set[idx].edge_index.cpu().t().numpy()
face_adj = np.hstack((np.full((face_adj.shape[0], 1), 2), face_adj))

f_rgba = h3d_set[idx].face_rgba.cpu().numpy() 
f_normal = h3d_set[idx].normals[:, 0:3].cpu().numpy()

mask = h3d_set[idx].y.cpu().numpy()
mask_rgba = np.array([h3d_set.rgba_dict[int(label)] for label in mask.tolist()], dtype=np.uint8)

## Mesh

In [ ]:
mesh.cell_data['RGBA'] = f_rgba
mesh.cell_data.active_scalars_name = 'RGBA'
mesh.plot(scalars='RGBA', 
          rgb=True, 
          cpos='iso', 
          window_size=w_size, 
          zoom=zoom,
          show_axes=False)

In [ ]:
mesh.cell_data['Mask'] = mask_rgba
mesh.cell_data.active_scalars_name = 'Mask'
mesh.plot(scalars='Mask', 
          rgb=True,
          cpos='iso', 
          window_size=w_size,
          zoom=zoom,
          show_edges=False,
          show_axes=False,
          show_scalar_bar=False)

In [ ]:
face_pcd = pv.PolyData(face_v)
face_pcd.lines = face_adj

face_pcd.point_data['RGBA'] = f_rgba
face_pcd.point_data.active_scalars_name = 'RGBA'
face_pcd.plot(scalars='RGBA', 
              rgb=True, 
              cpos='iso', 
              window_size=w_size, 
              zoom=zoom,
              render_points_as_spheres=True, 
              point_size=3, 
              line_width=1,
              show_axes=False)

In [ ]:
face_pcd.point_data['Mask'] = mask_rgba
face_pcd.point_data.active_scalars_name = 'Mask'
face_pcd.plot(scalars='Mask', 
              rgb=True,
              cpos='iso', 
              window_size=w_size, 
              zoom=zoom,
              render_points_as_spheres=True, 
              point_size=3, 
              line_width=1,
              show_axes=False,
              show_scalar_bar=False)

## Surface Normal

In [ ]:
face_pcd.point_data['Normal'] = f_normal
arrows = face_pcd.glyph(orient='Normal', scale=False, factor=0.05, tolerance=0.005)
arrows.plot(cpos='iso', 
            window_size=w_size,
            zoom=zoom,
            show_axes=False,
            show_scalar_bar=False)

## Prediction

In [ ]:
import yaml

from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torchmetrics.classification import Accuracy, F1Score, ConfusionMatrix

from train.trainer import Trainer
from model.net import LMSegNet, HGAPNet, LGAPNet, GANet


with open('cfg/h3d/lmseg_feature.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    cfg['device'] = 'cpu'
    cfg['path'] = 'save/h3d/lmseg_feature'
    cfg['epoch'] = '180'
    model = LMSegNet(cfg['in_channels'], 
                     cfg['out_channels'],
                     cfg['hid_channels'], 
                     cfg['num_convs'], 
                     cfg['pool_factors'], 
                     cfg['num_nbrs'],
                     cfg['num_block'],
                     cfg['alpha'], 
                     cfg['beta'])

trainer = Trainer(cfg=cfg) 
model = trainer.load_weights(model, f"epoch{cfg['epoch']}.pth")

## Evaluation

In [ ]:
h3d_set = H3DDataset(root='data/H3DBenchmark/', split='val', epoch='Epoch_March2018')
test_loader = DataLoader(h3d_set, 
                         batch_size=cfg['batch'], 
                         shuffle=False, 
                         num_workers=cfg['workers'])
metric_dict = {
    'OA': Accuracy(
        task="multiclass", 
        num_classes=cfg['out_channels'], 
        ignore_index=11,
        average='micro'
    ),
    'mF1': F1Score(
        task="multiclass", 
        num_classes=cfg['out_channels'], 
        ignore_index=11,
        average='macro'
    ),
    'F1': F1Score(
        task="multiclass", 
        num_classes=cfg['out_channels'], 
        ignore_index=11,
        average=None
    ),
}

cm = trainer.eval(model, 
                  test_loader, 
                  metric=metric_dict, 
                  ckpt=f"epoch{cfg['epoch']}.pth",
                  verbose=True)

In [ ]:
with torch.no_grad():
    y = model(Batch.from_data_list([h3d_set[idx]]).to(cfg['device']))['y'].detach().cpu().numpy()
    y = np.argmax(y, axis=-1, keepdims=False)

pred = np.array([h3d_set.rgba_dict[int(pred)] for pred in y.tolist()], dtype=np.uint8) 

mesh = pv.read(h3d_set.processed_mesh_list[idx])
mask = h3d_set[idx].y.cpu().numpy()

mesh.cell_data['Pred'] = pred
mesh.cell_data.active_scalars_name = 'Pred'
mesh.plot(scalars='Pred', 
          rgb=True, 
          cpos='iso', 
          window_size=w_size,           
          zoom=zoom,
          show_axes=False,
          show_scalar_bar=False)

In [ ]:
error = (y != mask)
error[mask == 11] = 0

mesh.cell_data['Error'] = error
mesh.cell_data.active_scalars_name = 'Error'
mesh.plot(scalars='Error', 
          cpos='iso', 
          cmap=['white', 'red'],
          window_size=w_size,           
          zoom=zoom,
          show_axes=False,
          show_scalar_bar=False)

In [ ]:
activations = {}

def hook_encoder(name):
    def _hook(mod, inp, out):
        pos_down, x_down, batch_down, edge_index_down = out
        activations[f'{name}/pos_down']   = [p.detach().cpu().numpy() for p in pos_down]
        activations[f'{name}/batch_down'] = [b.detach().cpu().numpy() for b in batch_down]
        activations[f'{name}/edge_down']  = [e.detach().cpu().numpy() for e in edge_index_down]
    return _hook

def hook_tensor(name):
    def _hook(mod, inp, out):
        activations[name] = out.detach().cpu().numpy()  # [N_pool, d]
    return _hook

In [ ]:
model.eval()
with torch.no_grad():
    out = model(Batch.from_data_list([h3d_set[idx]]).to(cfg['device']))
    Z = out['y'].detach().cpu()  # [N, C] logits

P = F.softmax(Z, dim=-1).numpy()
entropy = -(P * (np.log(P + 1e-12))).sum(axis=1)
# Normalize for display
emin, emax = entropy.min(), entropy.max()
entropy_norm = (entropy - emin) / (emax - emin + 1e-12)

mesh.cell_data['Entropy'] = entropy_norm
mesh.cell_data.active_scalars_name = 'Entropy'

mesh.plot(
    scalars='Entropy',
    cpos='iso',
    window_size=(1000, 1150), 
    zoom=zoom,
    cmap='viridis',            # perceptual; high=uncertain
    show_axes=False,
    show_scalar_bar=True,
    scalar_bar_args={
        "title": "Uncertainty (Entropy)",
        "position_x": 0.2,   # horizontal placement (0 = left, 1 = right)
        "position_y": 0.0,   # vertical placement (lower values push bar down),
        "title_font_size": 20,
        "label_font_size": 16,
        "italic": False,                
        "bold": True 
    }
)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import numpy as np

def pca_whiten(X, n_components=32, seed=0):
    """PCA -> whiten -> float32 features"""
    pca = PCA(n_components=n_components, whiten=True, random_state=seed)
    return pca.fit_transform(np.asarray(X, dtype=np.float32))

def l2_normalize(X, eps=1e-12):
    n = np.linalg.norm(X, axis=1, keepdims=True)
    return X / (n + eps)

def idw_upsample(src_pos, src_feat, dst_pos, k=1, eps=1e-12, power=2.0):
    """Inverse distance weighted upsampling."""
    nbrs = NearestNeighbors(n_neighbors=k).fit(src_pos)
    idx = nbrs.kneighbors(dst_pos, return_distance=False)
    d2  = np.sum((dst_pos[:, None, :] - src_pos[idx])**2, axis=-1) + eps
    w   = 1.0 / (d2 ** (power/2.0))
    w   = w / (w.sum(axis=1, keepdims=True) + eps)
    feat_k = src_feat[idx]       # (Nd, k, d)
    return np.einsum('nk,nkd->nd', w, feat_k)

def kmeans_labels(X, n_clusters=8, seed=0):
    km = KMeans(n_clusters=n_clusters, n_init=10, random_state=seed)
    return km.fit_predict(X).astype(np.int32)

h_enc = model.Enc.register_forward_hook(hook_encoder('Enc'))
h_hga = model.Enc.down_convs_hier[-1].register_forward_hook(hook_tensor('HGA_deep'))
h_lga = model.Enc.down_convs_local[-1].register_forward_hook(hook_tensor('LGA_last'))

model.eval()
with torch.no_grad():
    _ = model(Batch.from_data_list([h3d_set[idx]]).to(cfg['device']))

pos_list   = activations['Enc/pos_down']
pos_orig   = pos_list[0]                   # (N0, 3)
pos_deep   = pos_list[-1]                  # (Nd, 3)
H_hga_pool = activations['HGA_deep']       # (Nd, d_h)
H_lga_pool = activations['LGA_last']       # (Nd, d_l)

# PCA-whiten
Z_hga = pca_whiten(H_hga_pool, n_components=32, seed=0)
Z_lga = pca_whiten(H_lga_pool, n_components=32, seed=0)

# L2 normalization
Z_hga = l2_normalize(Z_hga)
Z_lga = l2_normalize(Z_lga)

# K-means clustering
n_clusters = 8
labels_hga_pool = kmeans_labels(Z_hga, n_clusters=n_clusters, seed=0)
labels_lga_pool = kmeans_labels(Z_lga, n_clusters=n_clusters, seed=0)

# 4) Upsample back to original resolution
labels_hga_full = idw_upsample(pos_deep, labels_hga_pool[:, None], pos_orig, k=1).ravel().astype(np.int32)
labels_lga_full = idw_upsample(pos_deep, labels_lga_pool[:, None], pos_orig, k=1).ravel().astype(np.int32)

mesh.cell_data['HGA_clusters'] = labels_hga_full
mesh.cell_data['LGA_clusters'] = labels_lga_full

annotations_hga = {int(i): f"HGA-{i}" for i in range(n_clusters)}
annotations_lga = {int(i): f"LGA-{i}" for i in range(n_clusters)}

# plot
mesh.cell_data.active_scalars_name = 'HGA_clusters'
mesh.plot(
    scalars='HGA_clusters',
    cpos='iso',
    window_size=(900, 900),
    zoom=1.1,
    cmap='Set3',
    categories=True,
    annotations=annotations_hga,
    show_axes=False,
    show_scalar_bar=True,
    scalar_bar_args={"title": "", "position_x": 0.2, "position_y": 0.0,
                     "title_font_size": 20, "label_font_size": 16, "italic": False, "bold": True}
)

mesh.cell_data.active_scalars_name = 'LGA_clusters'
mesh.plot(
    scalars='LGA_clusters',
    cpos='iso',
    window_size=(900, 900),
    zoom=1.1,
    cmap='Set3',
    categories=True,
    annotations=annotations_lga,
    show_axes=False,
    show_scalar_bar=True,
    scalar_bar_args={"title": "", "position_x": 0.2, "position_y": 0.0,
                     "title_font_size": 20, "label_font_size": 16, "italic": False, "bold": True}
)

## Gen Aggr

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme(style="darkgrid", 
              palette="pastel")
sns.set_theme(rc={"figure.dpi": 250, 
                  'savefig.dpi': 250,
                  "axes.spines.right": False, 
                  "axes.spines.top": False})

In [ ]:
def flatten_all(arrs):
    return [np.asarray(a).reshape(-1) for a in arrs if a is not None]

def stack_long(module_name, mode, arrays):
    rows = []
    for li, a in enumerate(arrays, 1):
        if a is None: 
            continue
        flat = np.asarray(a).reshape(-1)
        rows += [{"Module": module_name, "Mode": mode, "Layer": li, "t": v} for v in flat]
    return rows

def plot_compact_split_violin(HGA_t_avg, LGA_t_avg, HGA_t_max, LGA_t_max):
    df = pd.DataFrame(
        stack_long("HGA+", "t=0.0", flatten_all(HGA_t_avg)) +
        stack_long("LGA+", "t=0.0", flatten_all(LGA_t_avg)) +
        stack_long("HGA+", "t=1.0", flatten_all(HGA_t_max)) +
        stack_long("LGA+", "t=1.0", flatten_all(LGA_t_max))
    )

    layers = sorted(df["Layer"].unique())
    x_order = []
    for l in layers:
        x_order += [f"L{l}\n(t=0.0)", f"L{l}\n(t=1.0)"]
    df["LayerMode"] = df.apply(lambda r: f"L{r['Layer']}\n({r['Mode']})", axis=1)
    df["LayerMode"] = pd.Categorical(df["LayerMode"], categories=x_order, ordered=True)

    sns.set_context("paper") 
    plt.figure(figsize=(9, 4))

    ax = sns.violinplot(
        data=df, x="LayerMode", y="t",
        hue="Module", split=True, inner="quartile", cut=0, linewidth=0.8
    )

    ax.set_xlabel("", fontsize=10, weight="bold")
    ax.set_ylabel("t", fontsize=10, weight="bold")
    ax.set_title("Learnable t by layer & module — HGA+ vs. LGA+", fontsize=11, weight="bold")

    # --- add vertical dotted lines between L1, L2, L3 groups ---
    num_per_layer = 2  # each layer has 2 entries (t=0.0, t=1.0)
    for i in range(1, len(layers)):  # after L1, after L2
        ax.axvline(i * num_per_layer - 0.5, color="gray", linestyle="--", linewidth=0.8)

    # legend styling
    leg = ax.legend(title="Module", fontsize=9, title_fontsize=9, frameon=False, loc="upper left")
    for t in leg.texts: 
        t.set_fontsize(9)

    plt.tight_layout(pad=0.4)
    plt.show()


HGA_t_avg = []

for name, param in model.named_parameters():
  if name in ['Enc.down_convs_hier.0.gen_aggr_avg.t',
              'Enc.down_convs_hier.1.gen_aggr_avg.t',
              'Enc.down_convs_hier.2.gen_aggr_avg.t']:
    HGA_t_avg.append(param.detach().cpu().numpy())

LGA_t_avg = []
for name, param in model.named_parameters():
  if name in ['Enc.down_convs_local.0.gen_aggr_avg.t',
              'Enc.down_convs_local.1.gen_aggr_avg.t',
              'Enc.down_convs_local.2.gen_aggr_avg.t']:
    LGA_t_avg.append(param.detach().cpu().numpy())

HGA_t_max = []

for name, param in model.named_parameters():
  if name in ['Enc.down_convs_hier.0.gen_aggr_max.t',
              'Enc.down_convs_hier.1.gen_aggr_max.t',
              'Enc.down_convs_hier.2.gen_aggr_max.t']:
    HGA_t_max.append(param.detach().cpu().numpy())

LGA_t_max = []
for name, param in model.named_parameters():
  if name in ['Enc.down_convs_local.0.gen_aggr_max.t',
              'Enc.down_convs_local.1.gen_aggr_max.t',
              'Enc.down_convs_local.2.gen_aggr_max.t']:
    LGA_t_max.append(param.detach().cpu().numpy())
    
    
plot_compact_split_violin(HGA_t_avg, LGA_t_avg, HGA_t_max, LGA_t_max)